# Prompt Templates

Um prompt para um modelo de linguagem é um conjunto de instruções ou entradas fornecidas por um usuário para guiar a resposta do modelo, ajudando-o a entender o contexto e gerar uma saída baseada em linguagem relevante e coerente, como responder a perguntas, completar frases ou participar de uma conversa.

In [ ]:
from langchain_openai.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
Responda a seguinte pergunta do usuário:
{pergunta}
""")

prompt_template

In [ ]:
prompt_template.format(pergunta="O que é um buraco negro?")

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
{pergunta}
""")

prompt_template.format(
  pergunta="O que é um buraco negro?",
  n_palavras=50,
)

### Valores **default**

Usando **partial_variables**

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
{pergunta}
""", partial_variables={"n_palavras": 50})

prompt_template.format(pergunta="O que é um buraco negro?")

## Composing prompts | Unindo múltiplos prompts

In [ ]:
from langchain_core.prompts import PromptTemplate

template_word_count = PromptTemplate.from_template("""
Responda a pergunta em até {n_palavras} palavras.
""")

template_lingua = PromptTemplate.from_template("""
Retorna a resposta em {lingua}.
""")

template_final = (
  template_word_count
  + template_lingua
  + 'Responda a pergunta seguinte seguindo as instruções: {pergunta}'
)

template_final.template

In [ ]:
prompt = template_final.format(pergunta="O que é um buraco negro?", lingua="inglês", n_palavras=10)

llm.invoke(prompt)

# Templates para Chat

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_template("Essa é a minha dúvida: {duvida}")
chat_template.format_messages(duvida="Quem sou eu?")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
  [
    ('system', 'Você é um assistente engraçado e se chama {nome_assistente}'),
    ('human', 'Olá, como vai?'),
    ('ai', 'Melhor agora! Como posso ajudá-lo?'),
    ('human', '{pergunta}')
  ]
)

chat_template

In [ ]:
chat_template.format_messages(
  nome_assistente="Asimo",
  pergunta="Qual é o seu nome?"
)

In [ ]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()

chat.invoke(chat_template.format_messages(
  nome_assistente="Asimo",
  pergunta="Qual é o seu nome?"
))

# Template de Few-Shot Prompting para LLM

In [ ]:
exemplos = [
    {"pergunta": "Quem viveu mais tempo, Muhammad Ali ou Alan Turing?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? 
Resposta intermediária: Muhammad Ali tinha 74 anos quando morreu. 
Pergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? 
Resposta intermediária: Alan Turing tinha 41 anos quando morreu. 
Então a resposta final é: Muhammad Ali 
""", 
    }, 
    {"pergunta": "Quando nasceu o fundador do craigslist?", 
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi o fundador do craigslist? 
Resposta intermediária: O craigslist foi fundado por Craig Newmark. 
Pergunta de acompanhamento: Quando nasceu Craig Newmark? 
Resposta intermediária: Craig Newmark nasceu em 6 de dezembro de 1952. 
Então a resposta final é: 6 de dezembro de 1952 
""", 
    }, 
    {"pergunta": "Quem foi o avô materno de George Washington?",
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi a mãe de George Washington? 
Resposta intermediária: A mãe de George Washington foi Mary Ball Washington. 
Pergunta de acompanhamento: Quem foi o pai de Mary Ball Washington? 
Resposta intermediária: O pai de Mary Ball Washington foi Joseph Ball. 
Então a resposta final é: Joseph Ball 
""", 
    },
    {"pergunta": "Os diretores de Jaws e Casino Royale são do mesmo país?", 
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem é o diretor de Jaws? 
Resposta Intermediária: O diretor de Jaws é Steven Spielberg. 
Pergunta de acompanhamento: De onde é Steven Spielberg? 
Resposta Intermediária: Estados Unidos. 
Pergunta de acompanhamento: Quem é o diretor de Casino Royale? 
Resposta Intermediária: O diretor de Casino Royale é Martin Campbell. 
Pergunta de acompanhamento: De onde é Martin Campbell? 
Resposta Intermediária: Nova Zelândia. 
Então a resposta final é: Não 
""",
    },
]

In [ ]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate(
  input_variables=["pergunta", "resposta"],
  template="Pergunta: {pergunta}\nResposta: {resposta}\n",
)

example_prompt.format(**exemplos[0])

In [ ]:
prompt = FewShotPromptTemplate(
  examples=exemplos,
  example_prompt=example_prompt,
  suffix="Pergunta: {input}",
  input_variables=["input"],
)

In [ ]:
print(prompt.format(input="Quem fez mais gols, Romário ou Pelé?"))

In [ ]:
llm.invoke(prompt.format(input="Quem fez mais gols, Romário ou Pelé?"))

# Templates de Few-Shot Prompting para Chat

In [ ]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()

In [ ]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate

example_prompt = ChatPromptTemplate.from_messages([
  ('human', '{pergunta}'),
  ('ai', '{resposta}'),
])

print(example_prompt.format_messages(**exemplos[0]))

In [ ]:
few_shot_template = FewShotChatMessagePromptTemplate(
  examples=exemplos,
  example_prompt=example_prompt,
)

prompt = ChatPromptTemplate.from_messages([
  few_shot_template,
  ("human", "{input}"),
])

few_shot_template.format_messages()